In [1]:
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)

if not dir1 in sys.path:
    sys.path.append(dir1)

os.chdir('..')



In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
X = pd.read_csv('data//TS2Vec//x_emb.csv')
y = pd.read_csv('data//TS2Vec//y.csv')
#X_train_baseline = pd.read_csv('data/X_train_baseline.csv')
#y_train_baseline = pd.read_csv('data/y_train_baseline.csv')
#X_test_baseline = pd.read_csv('data/X_test_baseline.csv')
#y_test_baseline = pd.read_csv('data/y_test_baseline.csv')


In [3]:
y = y['Close']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Сравнение моделей на датасете Baseline

In [4]:
X_train.drop(columns=['Datetime', 'Stock'], inplace=True)
X_test.drop(columns=['Datetime', 'Stock'], inplace=True)
X_train.head()

,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,...,emb_118,emb_119,emb_120,emb_121,emb_122,emb_123,emb_124,emb_125,emb_126,emb_127
2406,-1775.40330,-623.976700,-1286.49580,-2043.458500,-2998.24930,-1149.48510,-2800.19600,-1645.508000,-1138.447500,-1584.72130,...,-1561.3536,-4227.08000,100.09961,-1513.04260,-3030.8470,-891.88550,282.163000,-4145.51100,-1586.297100,2321.48140
809,-163.95435,-73.424286,-244.87286,-65.485870,-257.21738,-235.31561,-287.58295,-191.254150,-401.178650,-381.17615,...,-693.5428,-49.10849,-431.54940,-308.53262,-602.7971,-270.95100,-379.211900,-285.04395,92.464650,-267.76202
1666,-313.37213,-186.837920,-231.81058,71.910446,-527.44050,-505.72876,-261.26960,110.797040,49.325874,-321.86496,...,-464.5166,-326.89246,71.84677,-365.96317,-348.7978,-253.60605,-15.205475,-549.68470,65.888596,334.06583
2680,-446.49893,-923.865600,-860.91050,-223.966100,-1758.70120,-1436.04820,-961.34973,42.782043,374.374700,-1255.54590,...,-1219.5334,-534.84436,-514.45160,-583.50550,-1271.6443,-923.97560,672.562400,-2347.82740,556.309400,782.70350
682,-1609.66980,-2877.516000,-1599.81730,174.296810,-3468.15100,-2903.25660,-3204.67870,-569.970700,73.829700,-2021.47340,...,1452.9744,-4825.97100,1265.57750,-2632.92900,-4533.0660,-2250.33500,1092.286900,-5629.20700,442.373600,4736.69900


In [5]:
from sklearn.metrics import mean_squared_error
def calculate_rmse(true_values, predicted_values):
    return np.sqrt(mean_squared_error(true_values, predicted_values))

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from catboost import CatBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

list_of_models = {'LinearRegression' : {'positive' : [True, False], 'fit_intercept' : [True, False]},
                  'RandomForestRegressor' : {'max_depth' : np.arange(2, 30, 4), 
                                             'min_samples_split' : np.arange(2, 60, 6), 
                                             'min_samples_leaf' : np.arange(2, 60, 6), 
                                             'criterion' : ['squared_error', 'absolute_error', 'friedman_mse', 'poisson']},

                  'KNeighborsRegressor' : {'n_neighbors' : np.arange(2, 100, 2), 
                                           'weights' : ['uniform', "distance"], 
                                           'metric' : ['cosine', 'euclidean', 'manhattan']},
                  
                  'CatBoostRegressor' : {'learning_rate': np.arange(0.01, 0.5, 0.002),
                                         'depth':  np.arange(1, 16, 1),
                                         'l2_leaf_reg': np.arange(3, 40, 2),
                                         'iterations': np.arange(30, 1000, 70),
                                         'loss_function': ['RMSE', 'MAE'],
                                        },
                    'SVR' : {'kernel': ['linear', 'poly', 'rbf'],
                            'C': np.arange(0.1, 40, 0.3),
                            'gamma': ['scale', 'auto'],
                            'shrinking': [True, False],
                        },
                    'Lasso' : {'alpha' : np.arange(0.1, 200, 0.5), 'fit_intercept' : [True, False]},
                    'Ridge' : {'alpha' : np.arange(0.1, 200, 0.5), 'fit_intercept' : [True, False]}
                
                    }

In [14]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression

def fit_tuning_predict(name_model, list_models, X_train, y_train, X_test, y_test):
    model = name_model()
    param_grid = list_models[name_model.__name__]
    

    grid_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=10, cv=5, n_jobs=6, verbose=2, scoring='neg_mean_absolute_error')
    random_model = grid_search.fit(X_train, y_train)
    best_params = random_model.best_params_

    #print(f"Лучшие гиперпараметры {name_model.__name__}: {best_params}")
    best_model = name_model(**best_params)
    best_model.fit(X_train, y_train)
    
    y_pred_fit = best_model.predict(X_train)
    rmse_fit = calculate_rmse(y_train, y_pred_fit)
    mae_fit = mean_absolute_error(y_train, y_pred_fit)

    y_pred_test = best_model.predict(X_test)
    rmse_test = calculate_rmse(y_test, y_pred_test)
    mae_test = mean_absolute_error(y_test, y_pred_test)
    return name_model.__name__ , rmse_fit, mae_fit, rmse_test, mae_test

    

In [15]:
result = []
for model_class in list_of_models:
    model_name = globals()[model_class]
    metrics = fit_tuning_predict(globals()[model_class], list_of_models, X_train, y_train, X_test, y_test)
    result.append(metrics)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [10]:
result_df = pd.DataFrame(result, columns=['Модель', 'train RMSE', ' train MAE', 'test RMSE', 'test MAE'])

In [11]:
result_df

,Модель,train RMSE,train MAE,test RMSE,test MAE
0,LinearRegression,166.906039,132.720356,164.963594,132.487313
1,RandomForestRegressor,79.467797,55.318541,88.068869,61.044302
2,KNeighborsRegressor,0.000234,0.000101,94.299389,67.910734
3,CatBoostRegressor,4.945993,3.763740,80.032185,55.475526
4,SVR,95.604045,57.324749,96.183676,58.452639
5,Lasso,86.106658,62.350647,88.546680,62.151767
6,Ridge,110.988522,88.966509,125.580520,93.461508


## Сравнение моделей на датасете Baseline с процентным изменением

In [ ]:
X_pct = pd.read_csv('data//TS2Vec//x_emb_pct_change.csv')
y_pct = pd.read_csv('data//TS2Vec//y_pct_change.csv')

y_pct = y_pct['Close']
X_train_pct, X_test_pct, y_train_pct, y_test_pct = train_test_split(X_pct, y_pct, test_size=0.2, random_state=42)

In [ ]:
X_train_pct.drop(columns=['Datetime', 'Stock'], inplace=True)
X_test_pct.drop(columns=['Datetime', 'Stock'], inplace=True)
X_train_pct.head()

In [ ]:
#X_train_baseline_t2v = pd.read_csv('data/X_train_baseline_t2v.csv')
#y_train_baseline_t2v = pd.read_csv('data/y_train_baseline_t2v.csv')
#X_test_baseline_t2v = pd.read_csv('data/X_test_baseline_t2v.csv')
#y_test_baseline_t2v = pd.read_csv('data/y_test_baseline_t2v.csv')

In [ ]:
result = []
for model_class in list_of_models:
    model_name = globals()[model_class]
    metrics = fit_tuning_predict(globals()[model_class], list_of_models, X_train_pct, y_train_pct, X_test_pct, y_test_pct)
    result.append(metrics)

In [54]:
result_df_t2v = pd.DataFrame(result, columns=['Модель', 'train RMSE', ' train MAE', 'test RMSE', 'test MAE'])

In [55]:
result_df_t2v

,Модель,train RMSE,train MAE,test RMSE,test MAE
0,LinearRegression,1.073485,0.624673,1.111537,0.816662
1,RandomForestRegressor,1.991700,0.832709,1.977375,1.091549
2,KNeighborsRegressor,3.665317,1.818426,5.285250,2.925881
3,CatBoostRegressor,1.429432,0.993610,4.419500,2.712931
4,SVR,49.382922,42.473867,47.682660,41.825219
5,Lasso,1.711525,0.871290,1.593986,0.866716
6,Ridge,1.507395,0.581805,1.213585,0.528903


In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Обучение модели линейной регрессии
linear_reg = LinearRegression()
linear_reg.fit(X_train_baseline, y_train_baseline)
linear_reg_predictions = linear_reg.predict(X_test_baseline)
linear_reg_mae = mean_absolute_error(y_test_baseline, linear_reg_predictions)
linear_reg_rmse = calculate_rmse(y_test_baseline, linear_reg_predictions)

# Обучение модели случайного леса
random_forest_reg = RandomForestRegressor()
random_forest_reg.fit(X_train_baseline, y_train_baseline)
random_forest_predictions = random_forest_reg.predict(X_test_baseline)
random_forest_mae = mean_absolute_error(y_test_baseline, random_forest_predictions)
random_forest_rmse = calculate_rmse(y_test_baseline, random_forest_predictions)

print("MAE линейной регрессии:", linear_reg_mae)
print("RMSE линейной регрессии:", linear_reg_rmse)
print('----')
print("MAE случайного леса:", random_forest_mae)
print("RMSE случайного леса:", random_forest_rmse)

MAE линейной регрессии: 0.5285833691909826
RMSE линейной регрессии: 1.2203221275386624
----
MAE случайного леса: 1.0196187924775344
RMSE случайного леса: 1.777853113770554


In [5]:
from sklearn.neighbors import KNeighborsRegressor
from catboost import CatBoostRegressor

# Обучение модели KNN
knn_reg = KNeighborsRegressor()
knn_reg.fit(X_train_baseline, y_train_baseline)
knn_predictions = knn_reg.predict(X_test_baseline)
knn_mae = mean_absolute_error(y_test_baseline, knn_predictions)
knn_rmse = calculate_rmse(y_test_baseline, knn_predictions)
# Обучение модели CatBoost
catboost_reg = CatBoostRegressor()
catboost_reg.fit(X_train_baseline, y_train_baseline, verbose=False)
catboost_predictions = catboost_reg.predict(X_test_baseline)
catboost_mae = mean_absolute_error(y_test_baseline, catboost_predictions)
catboost_rmse = calculate_rmse(y_test_baseline, catboost_predictions)
print("MAE для KNN:", knn_mae)
print("RMSE для KNN:", knn_rmse)
print('----')
print("MAE для CatBoost:", catboost_mae)
print("RMSE для CatBoost:", catboost_rmse)

MAE для KNN: 2.356628981438095
RMSE для KNN: 4.13112580932294
----
MAE для CatBoost: 2.576590102397551
RMSE для CatBoost: 4.595884427336685


In [9]:
from sklearn.svm import SVR
svr_model = SVR(kernel='linear')
svr_model.fit(X_train_baseline, y_train_baseline)
svr_predictions = svr_model.predict(X_test_baseline)
svr_mae = mean_absolute_error(y_test_baseline, svr_predictions)
svr_rmse = calculate_rmse(y_test_baseline, svr_predictions)
print("MAE для SVR:", svr_mae)
print("RMSE для SVR:", svr_rmse)

MAE для SVR: 0.5321849145407038
RMSE для SVR: 1.21571913112341


## Линейная регрессия показала наилучший результат на метриках, попробуем модифицировать её

In [27]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV

# Гиперпараметры для подбора
alphas = np.arange(0.01, 100, 0.5)

# Подбор гиперпараметра для гребневой регрессии
ridge = Ridge()
ridge_grid = GridSearchCV(ridge, {'alpha': alphas}, cv=5, scoring='neg_mean_absolute_error')
ridge_grid.fit(X_train_baseline, y_train_baseline)
best_alpha_ridge = ridge_grid.best_params_['alpha']
best_score_ridge = -ridge_grid.best_score_

# Подбор гиперпараметра для Лассо регрессии
lasso = Lasso()
lasso_grid = GridSearchCV(lasso, {'alpha': alphas}, cv=5, scoring='neg_mean_absolute_error')
lasso_grid.fit(X_train_baseline, y_train_baseline)
best_alpha_lasso = lasso_grid.best_params_['alpha']
best_score_lasso = -lasso_grid.best_score_

print("Лучший коэффициент регуляризации для гребневой регрессии:", best_alpha_ridge)
print("Лучший коэффициент регуляризации для Лассо регрессии:", best_alpha_lasso)


Лучший коэффициент регуляризации для гребневой регрессии: 99.51
Лучший коэффициент регуляризации для Лассо регрессии: 0.51


In [28]:
# Выбор лучшей модели гребневой регрессии с найденным коэффициентом
best_ridge_model = Ridge(alpha=best_alpha_ridge)
best_ridge_model.fit(X_train_baseline, y_train_baseline)
ridge_predictions_test = best_ridge_model.predict(X_test_baseline)
ridge_mae_test = mean_absolute_error(y_test_baseline, ridge_predictions_test)
ridge_rmse_test = calculate_rmse(y_test_baseline, ridge_predictions_test)

# Выбор лучшей модели Лассо регрессии с найденным коэффициентом
best_lasso_model = Lasso(alpha=best_alpha_lasso)
best_lasso_model.fit(X_train_baseline, y_train_baseline)
lasso_predictions_test = best_lasso_model.predict(X_test_baseline)
lasso_mae_test = mean_absolute_error(y_test_baseline, lasso_predictions_test)
lasso_rmse_test = calculate_rmse(y_test_baseline, lasso_predictions_test)

print("MAE для гребневой регрессии на тестовых данных:", ridge_mae_test)
print("RMSE для гребневой регрессии на тестовых данных:", ridge_rmse_test)
print("MAE для Лассо регрессии на тестовых данных:", lasso_mae_test)
print("RMSE для Лассо регрессии на тестовых данных:", lasso_rmse_test)


MAE для гребневой регрессии на тестовых данных: 0.5289001074778283
RMSE для гребневой регрессии на тестовых данных: 1.2223291040957456
MAE для Лассо регрессии на тестовых данных: 0.5208523141030297
RMSE для Лассо регрессии на тестовых данных: 1.212113292387616


In [25]:
import optuna
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split


def objective(trial):

    alpha = trial.suggest_loguniform('alpha', 1e-10, 2)
    max_iter = trial.suggest_int('max_iter', 100, 10000)  # Максимальное количество итераций
    tol = trial.suggest_loguniform('tol', 1e-6, 1e-1)  # Порог сходимости
    selection = trial.suggest_categorical('selection', ['cyclic', 'random'])  # Метод выбора признаков
    warm_start = trial.suggest_categorical('warm_start', [True, False])  # Перезапуск обучения
    

    model = Lasso(alpha=alpha)
    model.fit(X_train_baseline, y_train_baseline)
    

    y_pred = model.predict(X_test_baseline)
    
    # Вычисление MAE
    mae = mean_absolute_error(y_test_baseline, y_pred)
    
    return mae


study = optuna.create_study(direction='minimize')  # Минимизируем MAE
study.optimize(objective, n_trials=1000)  # Количество попыток оптимизации

# Вывод результатов
print('Лучшие значения гиперпараметров:', study.best_params)
print('Лучшее значение MAE:', study.best_value)


[I 2024-04-24 14:27:51,310] A new study created in memory with name: no-name-888ab3e1-f542-4b39-b9ec-dedfec46ecf7
[I 2024-04-24 14:27:51,391] Trial 0 finished with value: 0.5255058384151662 and parameters: {'alpha': 0.00016422644546428033, 'max_iter': 4226, 'tol': 0.0031712898747905506, 'selection': 'random', 'warm_start': True}. Best is trial 0 with value: 0.5255058384151662.
[I 2024-04-24 14:27:51,459] Trial 1 finished with value: 0.5255136472873913 and parameters: {'alpha': 4.4198661749300135e-10, 'max_iter': 2834, 'tol': 0.00018764180009444312, 'selection': 'cyclic', 'warm_start': True}. Best is trial 0 with value: 0.5255058384151662.
[I 2024-04-24 14:27:51,525] Trial 2 finished with value: 0.5255117202860806 and parameters: {'alpha': 4.052661785984517e-05, 'max_iter': 4038, 'tol': 0.001506991545553623, 'selection': 'random', 'warm_start': True}. Best is trial 0 with value: 0.5255058384151662.
[I 2024-04-24 14:27:51,595] Trial 3 finished with value: 0.5255099668385794 and parameter

Лучшие значения гиперпараметров: {'alpha': 0.08659943000652041, 'max_iter': 9103, 'tol': 0.04424039498493482, 'selection': 'cyclic', 'warm_start': False}
Лучшее значение MAE: 0.5207955617491807
